In [1]:
%%bash
pip install --upgrade duckdb
pip install --upgrade jupysql
pip install --upgrade duckdb-engine


Couldn't find program: 'bash'


In [2]:
import duckdb
import pandas as pd

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

#%sql duckdb:////content/dmeyf.db
%sql duckdb:///:default:


The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [3]:
%%sql
create or replace table competencia_03 as
select
    *
from read_csv_auto('~/buckets/b1/datasets/competencia_03_crudo.csv.gz')


,Success


In [4]:
%%sql
create or replace table targets as
with periodos as (
    select distinct foto_mes from competencia_03
), clientes as (
    select distinct numero_de_cliente from competencia_03
), todo as (
    select numero_de_cliente, foto_mes from clientes cross join periodos
), clase_ternaria as (
    select
        t.numero_de_cliente
        , t.foto_mes
        , if(c.numero_de_cliente is null, 0, 1) as mes_0
        , lead(mes_0, 1) over (partition by t.numero_de_cliente order by foto_mes) as mes_1
        , lead(mes_0, 2) over (partition by t.numero_de_cliente order by foto_mes) as mes_2
        , CASE
            WHEN mes_1 = 0 THEN 'BAJA+1'
            WHEN mes_1 = 1 AND mes_2 = 0 THEN 'BAJA+2'
            WHEN mes_1 = 1 AND mes_2 = 1 THEN 'CONTINUA'
            ELSE NULL
          END as clase_ternaria
    from todo t
    left join competencia_03 c using (numero_de_cliente, foto_mes)
) select
  foto_mes
  , numero_de_cliente
  , clase_ternaria
from clase_ternaria where mes_0 = 1


,Success


In [5]:
%%sql
alter table competencia_03 add column clase_ternaria VARCHAR(10)


,Success


In [6]:
%%sql
update competencia_03
set clase_ternaria = targets.clase_ternaria
from targets
where competencia_03.numero_de_cliente = targets.numero_de_cliente and competencia_03.foto_mes = targets.foto_mes;


,Success


In [7]:
%%sql
copy competencia_03 to '~/buckets/b1/datasets/competencia_03_preFE.csv.gz' (FORMAT CSV, HEADER)


RuntimeError: (duckdb.IOException) IO Error: Failed to delete file ".tmp\duckdb_temp_storage-1.tmp": El sistema no puede encontrar el archivo especificado.

[SQL: copy competencia_02 to 'G:/Mi unidad/Maestria Data Mining/2023_DMEyF/datasets/competencia_02.csv.gz' (FORMAT CSV, HEADER)]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
If you need help solving this issue, send us a message: https://ploomber.io/community
